# Scraping Winemag with scrapy

## A prove of concept

As a prove of concept, we scrape all reviews of wines from California. We separate the observations into some arbitrary groups via the start urls, because we can only retrieve 50,000 observations at once. The links column of the dataset that we create could be piped into another parse function to retrieve the full review of each individual wine. Also, we are using a dictionary to hold the entries, using lists instead would be more efficient, but this approach renders more readable text for now.

### Define Spider

In [1]:
from scrapy import Spider
from scrapy.http import Request
import re

class WinemagSpider(Spider):
    name = 'WinemagSpider'
    allowed_domains = ['www.winemag.com']
    start_urls = ['https://www.winemag.com/?s=&drink_type=wine&country=US&state=California&region=Central%20Coast&page=1', 
                  'https://www.winemag.com/?s=&drink_type=wine&country=US&state=California&region=Sonoma&page=1', 
                  'https://www.winemag.com/?s=&drink_type=wine&country=US&state=California&region=Napa&page=1', 
                  'https://www.winemag.com/?s=&drink_type=wine&country=US&state=California&region=California%20Other&page=1', 
                  'https://www.winemag.com/?s=&drink_type=wine&country=US&state=California&region=Mendocino%20County&page=1', 
                  'https://www.winemag.com/?s=&drink_type=wine&country=US&state=California&region=Sierra%20Foothills&page=1', 
                  'https://www.winemag.com/?s=&drink_type=wine&country=US&state=California&region=Napa-Sonoma&page=1', 
                  'https://www.winemag.com/?s=&drink_type=wine&country=US&state=California&region=Central%20Valley&page=1', 
                  'https://www.winemag.com/?s=&drink_type=wine&country=US&state=California&region=North%20Coast&page=1', 
                  'https://www.winemag.com/?s=&drink_type=wine&country=US&state=California&region=Lake%20County&page=1', 
                  'https://www.winemag.com/?s=&drink_type=wine&country=US&state=California&region=South%20Coast&page=1', 
                  'https://www.winemag.com/?s=&drink_type=wine&country=US&state=California&appellation=California-Washington&page=1', 
                  'https://www.winemag.com/?s=&drink_type=wine&country=US&state=California&appellation=California-Oregon&page=1', 
                  'https://www.winemag.com/?s=&drink_type=wine&country=US&state=California&appellation=Napa-Monterey-Mendocino&page=1', 
                  'https://www.winemag.com/?s=&drink_type=wine&country=US&state=California&appellation=Santa%20Barbara%20County-Sonoma%20County-Monterey%20County&page=1',
                  'https://www.winemag.com/?s=&drink_type=wine&country=US&state=California&region=Mendocino/Lake%20Counties&page=1']

    global wineries 
    wineries = []
    
    def parse(self, response):
        
        organizations = response.xpath('//*[@class="review-item "]')
        for organization in organizations:
            title = organization.xpath('.//*[@class="title"]/text()').extract_first()
            appelation = organization.xpath('.//*[@class="appellation"]//text()').extract_first()
            excerpt = organization.xpath('.//*[@class="excerpt"]//text()').extract_first()
            rating = organization.xpath('.//*[@class="rating"]//text()').extract_first()
            price = organization.xpath('.//*[@class="price"]//text()').extract_first()
            referer = response.url
            link = organization.xpath('.//a/@href').extract_first()
            
            wineries.append({'title': title, 'appelation': appelation, 'excerpt': excerpt, 
                             'rating': rating, 'price': price, 'referer': referer, 'link': link})
        
        page_n = int(re.search(r'\d+$', response.url).group())
        next_pages = response.xpath('//*[@class="pagination"]//a/text()').extract()
                
        if page_n == 1 and next_pages:
            max_page = int(next_pages[-1])
            
            url = re.search(r'.*[^\d+$]', response.url).group()
            
            for i in range(2, max_page + 1):
                absolute_url = url + str(i)
                yield Request(absolute_url, callback=self.parse)


### Initiate process and crawl website

In [2]:
from scrapy.crawler import CrawlerProcess
from datetime import date

process = CrawlerProcess({
    'USER_AGENT': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:67.0) Gecko/20100101 Firefox/67.0', 
    'DOWNLOAD_DELAY': '0.05', 
#     'CONCURRENT_REQUESTS': 2,
    'LOG_FILE': 'log_{}.txt'.format(date.today().isoformat())
})

In [3]:
process.crawl(WinemagSpider)

In [4]:
process.start()

In [5]:
process.stop()

<DeferredList at 0x7fd99c10df60 current result: []>

### Verify results

In [6]:
import pandas as pd

wineries = pd.DataFrame(wineries)
len(wineries)

77846

In [7]:
wineries.head()

,appelation,excerpt,link,price,rating,referer,title
0,North Coast,Delicious and exotic fruit flavors light up th...,https://www.winemag.com/buying-guide/de-loach-...,$45,93,https://www.winemag.com/?s=&drink_type=wine&co...,De Loach 2016 Marin Pinot Noir (Marin County)
1,North Coast,This is a super-concentrated and richly tannic...,https://www.winemag.com/buying-guide/eclectic-...,$20,92,https://www.winemag.com/?s=&drink_type=wine&co...,Eclectic 2014 Charbono (Suisun Valley)
2,North Coast,"Generous aromas, warm and ripe fruit flavors a...",https://www.winemag.com/buying-guide/fulcrum-2...,$39,92,https://www.winemag.com/?s=&drink_type=wine&co...,Fulcrum 2017 On Point Winemaker's Selection Pi...
3,North Coast,"Well structured and admirably dry, this wine i...",https://www.winemag.com/buying-guide/briceland...,$27,90,https://www.winemag.com/?s=&drink_type=wine&co...,Briceland 2015 Ishi Pishi Ranch Zinfandel (Hum...
4,North Coast,A deep color and ripe black-cherry flavor make...,https://www.winemag.com/buying-guide/kuleto-es...,$24,90,https://www.winemag.com/?s=&drink_type=wine&co...,Kuleto Estate 2016 India Ink Red (Lake County-...


In [8]:
wineries.to_csv('winemag_archive_{}.csv'.format(date.today().isoformat()))